In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sat May 06 22:00:52 2017

@author: binoy
"""

import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
from cStringIO import StringIO


def pdfparser(data):

    fp = file(data, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()

    return data
    
with open('ken.txt','w') as f:
    f.write(pdfparser('Ken_resume (2).pdf'))

In [1]:
from __future__ import unicode_literals
from gensim.summarization import keywords
import pandas as pd 

import gensim
import logging
import nltk
import string
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
import unicodedata
import math
import tqdm
from IPython.display import clear_output

# -*- coding: utf-8 -*-
"""
Created on Sat May 06 21:03:42 2017

@author: binoy
"""

class MyCountVectorizer:
    def __init__(self, docs):
        self.corpus = self.normalize_corpus(docs)
        self.make_features()
       # self.make_matrix()
        
    def normalize_corpus(self, docs):    
        table = string.maketrans(string.punctuation, 
                                 len(string.punctuation) * ' ')
        norm_docs = []
        for doc_raw in docs:
            doc = filter(lambda x: x in string.printable, doc_raw)
            '''
            doc = ''
            for x in doc_raw:
                if x in string.printable:
                    doc += x
            '''
            doc = str(doc).translate(table).lower()
            norm_docs.append(doc)
        #self.corpus = norm_docs
        return norm_docs
        
    def make_features(self):
        ''' create vocabulary set from the corpus '''
        stopwords = nltk.corpus.stopwords.words('english')
        self.features = set()
        for doc in self.corpus:
            for word in doc.split():
                if word not in stopwords:
                    self.features.add(word)
        #self.features = set([word for doc in self.corpus for word in doc.split() if word not in stopwords])
        self.features = sorted(list(self.features))

    def make_matrix(self):
        self.matrix = []
        for doc in self.corpus:
            doc_vec = []
            for word in self.features:
                tf = self.term_freq(word, doc)
                doc_vec.append(tf)
            self.matrix.append(doc_vec)

    def term_freq(self, term, document):
        words = document.split()
        count = 0
        for word in words:
            if word == term:
                count += 1
        return count
    
    def print_matrix(self):
        for vec in self.matrix:
            print(vec)

    def get_matrix(self):
        return self.matrix
    
    def get_features(self):
        return self.features
    
    def get_density(self):
        ''' get the density (# of non-zero elements / # all elements )'''
        counter = 0
        total = 0
        for row in self.matrix:
            for item in row:
                if item != 0:
                    counter += 1
                total += 1
        return 1.0 * counter / total
        


class MyTfIdfVectorizer(MyCountVectorizer):
    ''' inherits from MyCountVectorizer'''

    def make_matrix(self):
        'overriding method'
        self.matrix = []
        for doc in self.corpus:
            doc_vec = []
            for word in self.features:
                tf = self.term_freq(word, doc)
                idf = self.inverse_document_freq(word)
                doc_vec.append(tf * idf)
            #self.matrix.append(doc_vec)
            total = sum(doc_vec)
            doc_vec_norm = [i/total for i in doc_vec]
            self.matrix.append(doc_vec_norm)

    def inverse_document_freq(self, term):
        doc_count = 0
        for document in self.corpus:
            term_count = self.term_freq(term, document)
            if term_count > 0:
                doc_count += 1
        return math.log( 1.0 * len(self.corpus) / doc_count)

def plot_mds(mean_vec):   
   from sklearn.manifold import MDS
    
   data = mean_vec
   mds = MDS(n_components=2, random_state=1)
   pos = mds.fit_transform(data)
   xs,ys = pos[:,0], pos[:,1]
   for x, y in zip(xs, ys):
       plt.scatter(x, y)
    #    plt.text(x, y, name)
    #pos2 = mds.fit_transform(model.infer_vector(resume))
    #xs2,ys2 = pos2[:,0], pos2[:,1]
   plt.scatter(xs[-1], ys[-1], c='Red', marker='+')
   plt.text(xs[-1], ys[-1],'resume')
   plt.suptitle('MDS')
   plt.grid()
   plt.savefig('distance_MDS_improved.png')
   plt.show()

def plot_pca(meav_vec):
    from sklearn.decomposition import PCA
    data = mean_vec
    pca = PCA(n_components=2) #, whiten=True
    X = pca.fit_transform(data)
    xs,ys =X[:,0], X[:,1]
    plt.scatter(X[:,0], X[:,1])
    plt.scatter(xs[-1], ys[-1], c='Red', marker='+')
    plt.text(xs[-1], ys[-1],'resume')
    plt.grid()
    plt.suptitle('PCA')
    plt.savefig('distance_PCA_improved.png')
    plt.show()

       



'''

# Logging code taken from http://rare-technologies.com/word2vec-tutorial/
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

#==================================
# WARNING: if your computer memory is small, this won't work.
#=================================

'''    


class Rank :
	"""
	Use this function to determine the appropriate ranking/score of each file.
	When instantiated, this class will first load the keywords file
	
	--
	:param list keyword_list	- list containing each keyword found in keyword_file
	:return Object Rank
	"""
	
	def __init__(self, keyword_list):
		self.keywords	= keyword_list		
		self.total_keys = len(self.keywords)
	
	def get_rank(self, text):
		"""
		Get the rank of the file based on total count of keywords found in the file
		contents.
		"""
		# set the initial rank and count to 0
		rank = count = 0
		
		# get the percentage that each keyword is worth
		word_percentage = round(float(100)/float(len(self.keywords)), 2)
		# iterate over list of keywords	
		for keyword in self.keywords :
			keyword, multiplier = self.get_multiplier(keyword)
			
			# was the keyword found in the file? increase overall percentage if true
			rank += word_percentage if keyword in text else 0
			
			# get the number of occurrences of the keyword in the file
			count +=  text.count( keyword ) * int( multiplier )
						
		print "rank  ","Count ", "done"
		return (rank,count)
			
	
	def get_multiplier(self, keyword):
		"""
		Split the keyword on multiplier delimiter if found. Otherwise provide 1 for multiplier
		"""
		
		multiplier 	= 1 
		# set the multiplier if found in the file
		if ' *' in keyword :
			keyword,multiplier = keyword.split(' *')
		
		return (keyword, multiplier)

def get_keyword(yo):
    try:
        yo = yo.encode("utf-8")
        yo = yo.decode("utf-8")
    except:
        yo = yo.decode("utf-8")
    yo = keywords(yo)
    yo =yo.split("\n")
    #ps = PorterStemmer()
   # for n,i in enumerate(yo):
   #     yo[n] = ps.stem(i)
    return yo

C:\Users\gli26\Anaconda2\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
#nltk.download('stopwords')
model = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin', 
    binary=True) 

In [3]:

with open('ken.txt','r') as f:
    resume = f.read()
my_resume = get_keyword(resume)

bonus = ["models", "modeling risk management", "trading","analysis", 'optimize','financial','models',\
             'development','python','statistics','frm','reinvestment rate','credit','data']

for i, word in enumerate(my_resume):
    if word in bonus:
        my_resume[i] = word + " *2"

'''
with open("my_resume_keyword.txt","w") as file:
    for i in my_resume:
        i =  i.encode("utf-8")
        file.writelines(i)
stopwords = nltk.corpus.stopwords.words('english')

'''


df =pd.read_csv('data.csv')
jd = df['job_summary'].tolist()
companies = df['company'].tolist()
positions = df['jobtitle'].tolist()
result = pd.DataFrame(columns=["company","jobtitle","jd","keywords","rank","count"])

for i in tqdm.tqdm(range(len(jd))):  
    j = get_keyword(jd[i])
    my_rank = Rank(my_resume)
    my_rank = my_rank.get_rank(j)
    result = result.append({"company":companies[i],"jobtitle":positions[i],"jd":jd[i],"keywords":j,"rank":my_rank[0],"count":my_rank[1]}, ignore_index=True)
    clear_output()
result = result.sort_values(by="rank",ascending=False)
result = result.reset_index()
result = result[:101]
result.to_csv("job_match.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 171/171 [00:55<00:00,  4.62it/s]


In [ ]:
with open('ken.txt','r') as f:
    resume = f.read()
my_resume = get_keyword(resume)
with open("my_resume_keyword.txt","w") as file:
    for i in my_resume:
        i =  i.encode("utf-8")
        file.writelines(i)